In [76]:
from datetime import datetime



import Helper_Functions_Portfolio as gskp
import polars as pl
import hvplot.polars

# 1. Modeling Volatility and VaR

## Data

Find the data file `spy_data.xlsx`.

* Use the returns on the S&P 500 (`SPY`) and 1-month T-bills (`^IRX`).

* Calculate the excess market returns, using the treasury rate as the risk-free rate.

In [77]:
data = ("../data/spy_data.xlsx")

total_returns = pl.read_excel(data,sheet_name="total returns")
prices = pl.read_excel(data,sheet_name="prices")
total_returns = total_returns.with_columns(
    Excess = pl.col("SPY") - pl.col("^IRX")
)

## 1.1 Historic VaR.
Starting at `Jan 2, 2001`, calculate the historic-based VaR, based on the expanding sample from the first date through `Dec 29, 2000`. 

Denote $\tilde{r}^{VaR, 0.05}_{t}$ as the estimate of the time-t VaR based on data through $t − 1$.

### Report
Report the items below, starting at , starting at `Jan 2, 2001`.

* Plot $\tilde{r}^{VaR, 0.05}_{t}$ over time.

* Report the frequency of periods in which $\tilde{r} < \tilde{r}^{VaR, 0.05}_{t}$. Compare this to the quantile of $.05$.

* What drawbacks do you see in this historic VaR?

#### Note
By historic VaR, we mean simply taking the 5th quantile for the historic sample up to time $t − 1$. Of course, a sample size that is not a multiple of 100 will require some interpolation to get a 5th quantile. Your statistical package should handle this fine.

In [78]:
import polars as pl
import numpy as np

# Create a list to store VaR for each expanding sample, starting with None for t=0
expanding_var = [None]

# Iterate over the dataset, computing expanding quantiles
for i in range(1, total_returns.shape[0]):
    # Calculate the 5th percentile (VaR) on the expanding sample up to time t-1
    expanding_sample = total_returns[:i]['Excess']
    expanding_var.append(expanding_sample.quantile(0.05))

# Add the calculated VaR values to your DataFrame
total_returns = total_returns.with_columns(
    VaR=pl.Series(expanding_var)  # No need to shift anymore
)

# Filter based on date and plot
historic_var = total_returns.filter(pl.col("date") >= pl.datetime(2001,1,2)).select(["date", "VaR"])

# Plotting the result
historic_var.hvplot.line(x="date", y="VaR")



:Curve   [date]   (VaR)

In [79]:
frequency_below_var = total_returns.filter(pl.col("Excess") < pl.col("VaR")).shape[0]
total_periods = total_returns.shape[0]
proportion_below_var = frequency_below_var / total_periods
print(f"Proportion of periods where Excess < VaR: {proportion_below_var:.2%}")



Proportion of periods where Excess < VaR: 5.79%


The drawback for this method is that by using historical VaR we are actually using insane drawback periods such as the COVID crash and the 08 crash. While those tail risks event arent impossible they arent exaclty 

## 1.2 Volatility
We will calculate a time-series of volatility estimates using a few different methods. For each, we use $\sigma_t$ to denote our estimate of the time-t return volatility.

#### Expanding Series

$$
\sigma^2_{t,expanding} = \frac{1}{t-1}\sum_{\tau = 1}^{t-1}\tilde{r}^2_{\tau}
$$



#### Rolling Window

$$
\sigma^2_{t,rolling} = \frac{1}{m}\sum_{l = 1}^{m}\tilde{r}^2_{t-l}
$$

Use $m=$`252`.


#### Exponentially Weighted Moving Average (EWMA)

Feel free to calculate the EWMA using the following recursion,
$$
\sigma^2_{t, EWMA} = \theta \sigma^2_{t-1, EWMA} + (1-\theta)\tilde{r}^2_{t-1}
$$

Rather than estimating $\theta$, simply use $\theta$ = 0.94, and initialize with 
$$\sigma_{t_0} = \frac{0.20}{\sqrt{252}}$$

### Report
Report the items below, starting at , starting at `Jan 2, 2001`.

* For each of these three methods, plot $\sigma_t$. (Plot the vol, not the variance.)

* For each method, calculate the 5th percentile, 1-day-ahead VaR. We use a slight simplification of the normal VaR formula, by dropping $\mu$ from that formula, and rounding the normal distribution z-score to -1.65.
$$\tilde{r}^{VaR, 0.05}_{t} = −1.65 \sigma_t$$

* For each of these vol-based VaR estimates, calculate the frequency of periods in which $\tilde{r} < \tilde{r}^{VaR, 0.05}_{t}$

* Compare and contrast your results among each other and relative to the historic method in the previous problem.

In [80]:
m=252
theta = .94


initial_variance = (0.20 / (252 ** 0.5)) 


df = total_returns.with_columns([
    (pl.col("Excess").pow(2).cum_sum() / pl.arange(1, pl.len() + 1)).alias("expanding_variance")
])


df = df.with_columns([
    pl.col("expanding_variance").sqrt().alias("expanding_volatility")
])


df = df.with_columns([
    (pl.col("Excess").pow(2).rolling_mean(window_size=m)).alias("rolling_variance")
])


df = df.with_columns([
    pl.col("rolling_variance").sqrt().alias("rolling_volatility")
])





df = df.with_columns([
    (pl.col("Excess").pow(2)).alias("squared_returns")
])

df = df.with_columns([
    pl.lit(initial_variance).alias("ewma_variance")
])


df = df.with_columns([
    ((theta * (pl.col("ewma_variance").shift(1).pow(2))) + ((1 - theta) * pl.col("squared_returns")))
    .fill_null(initial_variance)  # Handle the first value with initial variance
    .alias("ewma_variance")
])



df = df.with_columns([
    pl.col("ewma_variance").sqrt().alias("ewma_volatility")
])


df = df.filter((pl.col("date") >= pl.datetime(2001,1,2)))

In [81]:
df.hvplot.line(x="date",y=['ewma_volatility', 'rolling_volatility', 'expanding_volatility'], value_label='Volatility')

:NdOverlay   [Variable]
   :Curve   [date]   (Volatility)

In [56]:

df = df.with_columns([
    (-1.65 * pl.col("expanding_volatility")).alias("expanding_var"),
    (-1.65 * pl.col("rolling_volatility")).alias("rolling_var"),
    (-1.65 * pl.col("ewma_volatility")).alias("ewma_var")
])


In [57]:

df = df.with_columns([
    (pl.col("Excess") < pl.col("expanding_var")).alias("below_expanding_var"),
    (pl.col("Excess") < pl.col("rolling_var")).alias("below_rolling_var"),
    (pl.col("Excess") < pl.col("ewma_var")).alias("below_ewma_var")
])


expanding_var_frequency = df["below_expanding_var"].sum() / df.height
rolling_var_frequency = df["below_rolling_var"].sum() / df.height
ewma_var_frequency = df["below_ewma_var"].sum() / df.height

print(f"Expanding VaR frequency: {expanding_var_frequency:.2%}")
print(f"Rolling VaR frequency: {rolling_var_frequency:.2%}")
print(f"EWMA VaR frequency: {ewma_var_frequency:.2%}")


Expanding VaR frequency: 4.61%
Rolling VaR frequency: 5.20%
EWMA VaR frequency: 3.58%


The rolling value would be the most accurate and best

## 1.3 CVaR
Re-do the previous two problems, but this time calculating CVaR instead of VaR, (still for $q =$ `.05`.) 

In [69]:
import polars as pl
from scipy.stats import norm

# Parameters for CVaR
z_value = 1.65
phi_z = norm.pdf(z_value)  # PDF of normal distribution at 1.65
alpha = 0.05
cvar_multiplier = -phi_z / alpha  # This will be used for parametric CVaR

# Assuming 'std' columns contain the volatilities for each method
cvar_df = pl.DataFrame({
    "date": df["date"],
    "expanding_volatility": df["expanding_volatility"],
    "rolling_volatility": df["rolling_volatility"],
    "ewma_volatility": df["ewma_volatility"]
})

# Parametric CVaR formula application
cvar_df = cvar_df.with_columns([
    (cvar_multiplier * pl.col("expanding_volatility")).alias("expanding_cvar"),
    (cvar_multiplier * pl.col("rolling_volatility")).alias("rolling_cvar"),
    (cvar_multiplier * pl.col("ewma_volatility")).alias("ewma_cvar")
])

# Historical CVaR calculation using expanding window
expanding_cvar_list = []

for i in range(60, df.height):  # Start at 60 because it's an expanding window
    # Select data up to the current point (expanding)
    window_data = df[:i].select("Excess").to_series()  # Convert to Series for direct operations
    
    var_5_percentile = window_data.quantile(alpha)  # 5% VaR threshold
    
    # Filter returns below VaR
    below_var_returns = window_data.filter(window_data < var_5_percentile)
    
    # Calculate CVaR if there are returns below the VaR
    expanding_cvar = below_var_returns.mean() if len(below_var_returns) > 0 else None
    expanding_cvar_list.append(expanding_cvar)

# Align CVaR values with the original DataFrame
historic_cvar_series = [None] * 60 + expanding_cvar_list  # Pad the first 60 entries with None
cvar_df = cvar_df.with_columns([
    pl.Series(historic_cvar_series).alias("historic_cvar")
])

# Filter for date >= 2001-01-02
cvar_df = cvar_df.filter(pl.col("date") >= pl.datetime(2001, 1, 2))

# Plotting CVaR estimates
cvar_df.hvplot.line(x="date", y=["expanding_cvar", "rolling_cvar", "ewma_cvar", "historic_cvar"], title="CVaR Estimates", ylabel="5% CVaR", xlabel="Date")


:NdOverlay   [Variable]
   :Curve   [date]   (value)